#  검색 성능 향상을 위한 기법 

- 재순위화 (Re-rank)
- 맥락 압축  (Contextural Compression)

### **학습 목표:** 재순위화(Re-rank) 기법, 맥락 압축(Contextual Compression) 기법을 활용하여 최종 검색 시스템을 구축한다

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) langfuase handler 설정`

In [3]:
from langfuse.langchain import CallbackHandler

# LangChain 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

`(4) 벡터스토어 로드`

In [4]:
# 벡터 저장소 로드 
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

chroma_db = Chroma(
    collection_name="db_korean_cosine_metadata",
    embedding_function=embeddings,
    persist_directory="./chroma_db",
)

`(5) 백터 검색기 생성`

In [5]:
# 기본 retriever 초기화
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 5}
)

query = "테슬라 트럭 모델이 있나요?"
retrieved_docs = chroma_k_retriever.invoke(query, config={"callbacks": [langfuse_handler]})

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
- **Cybertruck:** 2019년 11월에 처음 발표된 풀사이즈 픽업 트럭. 후륜 구동, 듀얼 모터 전륜 구동, 트리 모터 전륜 구동의 세 가지 모델이 제공됩니다. [출처: data\테슬라_KR.md]
[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
| Tesla 모델 |          |       |                  |
| :--------- | :------- | :---- | :--------------- |
| 이름       | 제조년도 | 좌석  | 참고             |
| Roadster   | 2008     | 2     | 2012년에 단종    |
| Model S    | 2012     | 5/7   |                  |
| Model X    | 2015     | 5/6/7 |                  |
| Model 3    | 2017     | 5     |                  |
| Model Y    | 2020     | 5/7   |                  |
| Semi       | 2022     | 2     |                  |
| Cybertruck | 2023     | 5     |                  |
| Roadster 2 |          | 2/4   | 2025년 출시 예정 |
| Cybercab   |          | 2     | 2026년 출시 예정 |
| Robovan    |          | 20    | 명시된 기간 없음 |

### 사용 가능한 제품 [출처: data\테슬라_KR.md]
[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
- **Mod

---

## **Re-rank** (재순위화)

- **재순위화**는 검색 결과를 재분석하여 최적의 순서로 정렬하는 고도화된 기술임

- **이중 단계 프로세스**로 기본 검색 후 정교한 기준으로 재평가를 진행함
    1. 먼저 기본 검색 알고리즘으로 관련 문서들을 찾은 후, 
    2. 더 정교한 기준으로 이들을 재평가하여 최종 순위를 결정

- 사용자의 검색 의도에 맞는 **정확도 향상**을 통해 검색 품질을 개선함

- 검색 결과의 품질을 높이기 위한 체계적인 최적화 방법론

--- 
### 1) **Cross Encoder** Reranker

- **Cross-Encoder** 모델을 활용하여 검색 결과의 정밀한 재정렬을 수행함
- 데이터를 **쌍(pair) 단위**로 처리하여 문서와 쿼리 간의 관계를 분석함 (예: 두 개의 문장 또는 문서)
- **통합 인코딩 방식**으로 검색 쿼리와 검색된 문서 간 유사도를 더 정확하게 계산함

- 참고: https://www.sbert.net/examples/applications/cross-encoder/README.html

`(1) 모델 초기화`

In [6]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# CrossEncoderReranker 모델 초기화 
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# CrossEncoderReranker 모델을 사용한 re-ranker 초기화 (top_n: 3)
re_ranker = CrossEncoderReranker(model=model, top_n=3)

# CrossEncoderReranker를 사용한 retriever 초기화
cross_encoder_reranker_retriever = ContextualCompressionRetriever(
    base_compressor=re_ranker, 
    base_retriever=chroma_k_retriever,
)

`(2) 문서 검색`

In [7]:
# CrossEncoderReranker를 사용한 retriever를 사용하여 검색
query = "테슬라 트럭 모델이 있나요?"
retrieved_docs = cross_encoder_reranker_retriever.invoke(query, config={"callbacks": [langfuse_handler]})

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
- **Cybertruck:** 2019년 11월에 처음 발표된 풀사이즈 픽업 트럭. 후륜 구동, 듀얼 모터 전륜 구동, 트리 모터 전륜 구동의 세 가지 모델이 제공됩니다. [출처: data\테슬라_KR.md]
[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
- **Model Y:** 싱글 모터, 후륜 구동 또는 듀얼 모터, 전륜 구동 레이아웃을 갖춘 5인승 및 7인승 구성으로 제공되는 중형 크로스오버 SUV. 이 차량은 고급 Model X SUV보다 저렴하도록 설계되었습니다. Model Y 프로토타입은 2019년 3월에 처음 공개되었으며 배송은 2020년 3월에 시작되었습니다.
- **Tesla Semi:** Tesla Semi는 Tesla, Inc.의 클래스 8 세미 트럭으로, 트리 모터, 후륜 구동 레이아웃을 갖추고 있습니다. Tesla는 Semi가 일반적인 디젤 세미 트럭보다 약 3배 더 강력하고 주행 거리가 500마일(800km)이라고 주장합니다. 초기 배송은 2022년 12월 1일에 PepsiCo에 이루어졌습니다. [출처: data\테슬라_KR.md]
[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
| Tesla 모델 |          |       |                  |
| :--------- | :------- | :---- | :--------------- |
| 이름       | 제조년도 | 좌석  | 참고             |
| Roadster   | 2008     | 2     | 2012년에 단종    |
| Model S    | 2012     | 5/7   |                  |
| Model X    | 2015     | 5/6/7 |                

### 2) **LLM** Reranker

- **대규모 언어 모델**을 활용하여 검색 결과의 재순위화를 수행함
- 쿼리와 문서 간의 **관련성 분석**을 통해 최적의 순서를 도출함
- **LLMListwiseRerank**와 같은 전문화된 재순위화 모델을 적용함

`(1) 모델 초기화`

In [8]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMListwiseRerank
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# LLMListwiseRerank 모델 초기화 (top_n: 3)
re_ranker = LLMListwiseRerank.from_llm(llm, top_n=3)

# LLMListwiseRerank 모델을 사용한 re-ranker 초기화
llm_reranker_retriever = ContextualCompressionRetriever(
    base_compressor=re_ranker, 
    base_retriever=chroma_k_retriever,
)

`(2) 문서 검색`

In [9]:
# LLMListwiseRerank 모델을 사용한 retriever를 사용하여 검색

query = "테슬라 트럭 모델이 있나요?"
retrieved_docs = llm_reranker_retriever.invoke(query, config={"callbacks": [langfuse_handler]})


for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
- **Cybertruck:** 2019년 11월에 처음 발표된 풀사이즈 픽업 트럭. 후륜 구동, 듀얼 모터 전륜 구동, 트리 모터 전륜 구동의 세 가지 모델이 제공됩니다. [출처: data\테슬라_KR.md]
[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
- **Model Y:** 싱글 모터, 후륜 구동 또는 듀얼 모터, 전륜 구동 레이아웃을 갖춘 5인승 및 7인승 구성으로 제공되는 중형 크로스오버 SUV. 이 차량은 고급 Model X SUV보다 저렴하도록 설계되었습니다. Model Y 프로토타입은 2019년 3월에 처음 공개되었으며 배송은 2020년 3월에 시작되었습니다.
- **Tesla Semi:** Tesla Semi는 Tesla, Inc.의 클래스 8 세미 트럭으로, 트리 모터, 후륜 구동 레이아웃을 갖추고 있습니다. Tesla는 Semi가 일반적인 디젤 세미 트럭보다 약 3배 더 강력하고 주행 거리가 500마일(800km)이라고 주장합니다. 초기 배송은 2022년 12월 1일에 PepsiCo에 이루어졌습니다. [출처: data\테슬라_KR.md]
[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
| Tesla 모델 |          |       |                  |
| :--------- | :------- | :---- | :--------------- |
| 이름       | 제조년도 | 좌석  | 참고             |
| Roadster   | 2008     | 2     | 2012년에 단종    |
| Model S    | 2012     | 5/7   |                  |
| Model X    | 2015     | 5/6/7 |                

---

## **Contextual Compression** (맥락적 압축)

- **맥락적 압축 기술**은 검색된 문서를 그대로 반환하는 대신, 쿼리 관련 정보만을 선별적으로 추출함

- **이중 구조 시스템**으로 기본 검색과 문서 압축 과정을 수행함
    1. 기본 검색기(base retriever) 
    2. 문서 압축기(Document Compressor)

- **효율적인 처리**를 통해 LLM 비용 절감과 응답 품질 향상을 달성함

### 1) **LLMChainFilter**

- **LLM 기반 필터링**으로 검색된 문서의 포함 여부를 결정함
- **원본 유지 방식**으로 문서 내용의 변경 없이 선별 작업을 수행함
- **선택적 필터링**을 통해 관련성 높은 문서만을 최종 반환함
- 문서 원본을 보존하면서 관련성 기반의 스마트한 선별을 수행하는 방식

`(1) 모델 초기화`

In [10]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# LLMChainFilter 모델 초기화
context_filter = LLMChainFilter.from_llm(llm)

# LLMChainFilter 모델을 사용한 retriever 초기화
llm_filter_compression_retriever = ContextualCompressionRetriever(
    base_compressor=context_filter,                   # LLM 기반 압축기
    base_retriever=chroma_k_retriever,               # 기본 검색기 
)

`(2) 문서 검색`

In [11]:
# LLMListwiseRerank 모델을 사용한 retriever를 사용하여 검색

query = "테슬라 트럭 모델이 있나요?"
compressed_docs = llm_filter_compression_retriever.invoke(query, config={"callbacks": [langfuse_handler]})


for doc in compressed_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
- **Cybertruck:** 2019년 11월에 처음 발표된 풀사이즈 픽업 트럭. 후륜 구동, 듀얼 모터 전륜 구동, 트리 모터 전륜 구동의 세 가지 모델이 제공됩니다. [출처: data\테슬라_KR.md]
[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
| Tesla 모델 |          |       |                  |
| :--------- | :------- | :---- | :--------------- |
| 이름       | 제조년도 | 좌석  | 참고             |
| Roadster   | 2008     | 2     | 2012년에 단종    |
| Model S    | 2012     | 5/7   |                  |
| Model X    | 2015     | 5/6/7 |                  |
| Model 3    | 2017     | 5     |                  |
| Model Y    | 2020     | 5/7   |                  |
| Semi       | 2022     | 2     |                  |
| Cybertruck | 2023     | 5     |                  |
| Roadster 2 |          | 2/4   | 2025년 출시 예정 |
| Cybercab   |          | 2     | 2026년 출시 예정 |
| Robovan    |          | 20    | 명시된 기간 없음 |

### 사용 가능한 제품 [출처: data\테슬라_KR.md]
[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
- **Mod

### 2) **LLMChainExtractor**

- **LLM 기반 추출**로 문서에서 쿼리 관련 핵심 내용만을 선별함
- **순차적 처리 방식**으로 각 문서를 검토하여 관련 정보를 추출함
- **맞춤형 요약**을 통해 쿼리에 최적화된 압축 결과를 생성함
- 쿼리 맥락에 따른 선별적 정보 추출로 효율적인 문서 압축을 실현

`(1) 모델 초기화`

In [12]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# LLMChainExtractor 모델 초기화
compressor = LLMChainExtractor.from_llm(llm)

# LLMChainExtractor 모델을 사용한 retriever 초기화
llm_extractor_compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,                                    # LLM 기반 압축기
    base_retriever=cross_encoder_reranker_retriever,               # 기본 검색기 (Re-rank)
)

`(2) 문서 검색`

In [13]:
# LLMChainExtractor 모델을 사용한 retriever를 사용하여 검색

query = "테슬라 트럭 모델이 있나요?"
compressed_docs = llm_extractor_compression_retriever.invoke(query, config={"callbacks": [langfuse_handler]})


for doc in compressed_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

- **Cybertruck:** 2019년 11월에 처음 발표된 풀사이즈 픽업 트럭. 후륜 구동, 듀얼 모터 전륜 구동, 트리 모터 전륜 구동의 세 가지 모델이 제공됩니다. [출처: data\테슬라_KR.md]
- **Tesla Semi:** Tesla Semi는 Tesla, Inc.의 클래스 8 세미 트럭으로, 트리 모터, 후륜 구동 레이아웃을 갖추고 있습니다. Tesla는 Semi가 일반적인 디젤 세미 트럭보다 약 3배 더 강력하고 주행 거리가 500마일(800km)이라고 주장합니다. 초기 배송은 2022년 12월 1일에 PepsiCo에 이루어졌습니다. [출처: data\테슬라_KR.md]
| Tesla 모델 |          |       |                  |
| :--------- | :------- | :---- | :--------------- |
| Semi       | 2022     | 2     |                  |
| Cybertruck | 2023     | 5     |                  |
| Cybercab   |          | 2     | 2026년 출시 예정 | [출처: data\테슬라_KR.md]


### 3) **EmbeddingsFilter**

- **임베딩 기반 필터링**으로 문서와 쿼리 간 유사도를 계산함
- **LLM 미사용 방식**으로 빠른 처리 속도와 비용 효율성을 확보함 (LLM 호출보다 저렴하고 빠른 옵션)
- **유사도 기준 선별**을 통해 관련성 높은 문서만을 효과적으로 추출함
- 경제적이고 신속한 임베딩 기반의 문서 필터링 기법 

`(1) 모델 초기화`

In [14]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

# 임베딩 기반 압축기 초기화
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.4)

# 임베딩 기반 압축기를 사용한 retriever 초기화
embed_filter_compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter,                             # 임베딩 기반 압축기
    base_retriever=cross_encoder_reranker_retriever,               # 기본 검색기 (Re-rank)
)

`(2) 문서 검색`

In [15]:
query = "테슬라 트럭 모델이 있나요?"
compressed_docs = embed_filter_compression_retriever.invoke(query, config={"callbacks": [langfuse_handler]})

for doc in compressed_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
- **Cybertruck:** 2019년 11월에 처음 발표된 풀사이즈 픽업 트럭. 후륜 구동, 듀얼 모터 전륜 구동, 트리 모터 전륜 구동의 세 가지 모델이 제공됩니다. [출처: data\테슬라_KR.md]
[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
| Tesla 모델 |          |       |                  |
| :--------- | :------- | :---- | :--------------- |
| 이름       | 제조년도 | 좌석  | 참고             |
| Roadster   | 2008     | 2     | 2012년에 단종    |
| Model S    | 2012     | 5/7   |                  |
| Model X    | 2015     | 5/6/7 |                  |
| Model 3    | 2017     | 5     |                  |
| Model Y    | 2020     | 5/7   |                  |
| Semi       | 2022     | 2     |                  |
| Cybertruck | 2023     | 5     |                  |
| Roadster 2 |          | 2/4   | 2025년 출시 예정 |
| Cybercab   |          | 2     | 2026년 출시 예정 |
| Robovan    |          | 20    | 명시된 기간 없음 |

### 사용 가능한 제품 [출처: data\테슬라_KR.md]
[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
- **Mod

### 4) **DocumentCompressorPipeline**

- **파이프라인 구조**로 여러 압축기를 순차적으로 연결하여 처리함
- **복합 변환 기능**으로 문서 분할 및 중복 제거 등 다양한 처리가 가능함
- **유연한 확장성**을 통해 BaseDocumentTransformers 추가로 기능을 확장함
- 다중 압축기를 연계하여 포괄적이고 효과적인 문서 처리를 구현하는 방식

`(1) 모델 초기화`

In [16]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter


# 임베딩 기반 필터 초기화 - 중복 제거
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

# 임베딩 기반 필터 초기화 - 유사도 기반 필터 (임베딩 유사도 0.4 이상)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.4)

# Re-ranking 모델 초기화
re_ranker = LLMListwiseRerank.from_llm(llm, top_n=2)

# DocumentCompressorPipeline 초기화 (순차적으로 redundant_filter -> relevant_filter -> re_ranker 적용)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[redundant_filter, relevant_filter, re_ranker]
)

# DocumentCompressorPipeline을 사용한 retriever 초기화
pipeline_compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor,           # DocumentCompressorPipeline 기반 압축기
    base_retriever=chroma_k_retriever,             # 기본 검색기
)

`(2) 문서 검색`

In [17]:
query = "테슬라 트럭 모델이 있나요?"
compressed_docs = pipeline_compression_retriever.invoke(query, config={"callbacks": [langfuse_handler]})

for doc in compressed_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
- **Cybertruck:** 2019년 11월에 처음 발표된 풀사이즈 픽업 트럭. 후륜 구동, 듀얼 모터 전륜 구동, 트리 모터 전륜 구동의 세 가지 모델이 제공됩니다. [출처: data\테슬라_KR.md]
[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
- **Model Y:** 싱글 모터, 후륜 구동 또는 듀얼 모터, 전륜 구동 레이아웃을 갖춘 5인승 및 7인승 구성으로 제공되는 중형 크로스오버 SUV. 이 차량은 고급 Model X SUV보다 저렴하도록 설계되었습니다. Model Y 프로토타입은 2019년 3월에 처음 공개되었으며 배송은 2020년 3월에 시작되었습니다.
- **Tesla Semi:** Tesla Semi는 Tesla, Inc.의 클래스 8 세미 트럭으로, 트리 모터, 후륜 구동 레이아웃을 갖추고 있습니다. Tesla는 Semi가 일반적인 디젤 세미 트럭보다 약 3배 더 강력하고 주행 거리가 500마일(800km)이라고 주장합니다. 초기 배송은 2022년 12월 1일에 PepsiCo에 이루어졌습니다. [출처: data\테슬라_KR.md]


--- 
# **[실습]**

- 지금까지 학습한 여러 기법들을 선택하여, RAG 답변을 생성하는 체인을 구성합니다. 

`(1) 기본 검색기 설정`

- Semantic Search, Keyword Search, Hybrid Search 검색기를 직접 정의합니다. 
- 쿼리 확장 도구 적용을 검토합니다.

In [20]:
# 실습 1: 16코어 128GB 최적화 + 상세 시간 측정 검색기 설정 (Jupyter 호환)

import asyncio
import time
import os
import pickle
import psutil
import threading
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import lru_cache
from tqdm import tqdm
from pathlib import Path
from typing import List, Dict, Any

print("🚀 16코어 128GB 최적화 RAG 검색기 초기화 시작!")
print(f"💻 시스템 정보: CPU {os.cpu_count()}코어, 메모리 {psutil.virtual_memory().total // (1024**3)}GB")

# 16코어 128GB 시스템 최적화 병렬도 계산 (Jupyter 호환)
def get_jupyter_optimized_parallelism():
    """16코어 128GB 시스템 최적화 병렬도 (Jupyter 환경)"""
    cpu_cores = os.cpu_count()  # 16
    total_memory_gb = psutil.virtual_memory().total // (1024**3)  # 128
    
    # Jupyter 환경에 최적화된 병렬도 (ThreadPoolExecutor 중심)
    parallelism = {
        "file_loading": min(cpu_cores * 4, 32),      # I/O 집약적: 32개 스레드
        "text_splitting": min(cpu_cores * 2, 24),   # CPU 집약적: 24개 스레드 (Process 대신 Thread)
        "embedding": cpu_cores * 4,                  # API 호출: 64개 동시 요청
        "bm25_indexing": min(cpu_cores * 2, 24),    # 메모리 집약적: 24개 스레드
        "search_test": 8                             # 검색 테스트: 8개 동시
    }
    
    print(f"🔥 Jupyter 최적화 병렬도: {parallelism}")
    return parallelism

PARALLELISM = get_jupyter_optimized_parallelism()

# 캐싱 디렉토리 설정
CACHE_DIR = "./cache"
os.makedirs(CACHE_DIR, exist_ok=True)

# 상세 시간 측정 및 성능 모니터링 클래스
class DetailedPerformanceMonitor:
    def __init__(self):
        self.start_time = time.time()
        self.stage_times = {}
        self.stage_details = {}
        
    def log_stage_start(self, stage_name):
        """단계 시작 시간 기록"""
        start_time = time.time()
        start_datetime = datetime.now().strftime("%H:%M:%S.%f")[:-3]
        
        self.stage_times[stage_name] = {"start": start_time, "start_datetime": start_datetime}
        
        print(f"\n⏰ [{start_datetime}] 🚀 {stage_name} 시작")
        print(f"   💻 시작 시 시스템 상태: {self.get_detailed_system_stats()}")
        
        return start_time
    
    def log_stage_end(self, stage_name):
        """단계 종료 시간 기록"""
        end_time = time.time()
        end_datetime = datetime.now().strftime("%H:%M:%S.%f")[:-3]
        
        if stage_name in self.stage_times:
            start_time = self.stage_times[stage_name]["start"]
            duration = end_time - start_time
            
            self.stage_times[stage_name].update({
                "end": end_time,
                "end_datetime": end_datetime,
                "duration": duration
            })
            
            print(f"⏰ [{end_datetime}] ✅ {stage_name} 완료")
            print(f"   ⏱️  소요시간: {duration:.3f}초")
            print(f"   💻 종료 시 시스템 상태: {self.get_detailed_system_stats()}")
            print(f"   📊 성능 요약: {self.get_performance_summary(stage_name)}")
            
            return duration
        return 0
    
    def get_detailed_system_stats(self):
        """상세 시스템 상태 반환"""
        try:
            cpu_percent = psutil.cpu_percent(interval=0.1, percpu=True)
            memory = psutil.virtual_memory()
            
            return {
                "cpu_avg": f"{sum(cpu_percent)/len(cpu_percent):.1f}%",
                "cpu_cores": [f"C{i}:{cpu:.1f}%" for i, cpu in enumerate(cpu_percent[:8])],  # 처음 8코어만 표시
                "memory_used": f"{memory.used // (1024**3)}GB/{memory.total // (1024**3)}GB ({memory.percent:.1f}%)",
                "memory_available": f"{memory.available // (1024**3)}GB",
                "active_threads": threading.active_count()
            }
        except:
            return {"status": "모니터링 일시 중단"}
    
    def get_performance_summary(self, stage_name):
        """단계별 성능 요약"""
        if stage_name not in self.stage_times:
            return "측정 데이터 없음"
        
        duration = self.stage_times[stage_name]["duration"]
        
        if duration < 1:
            return f"⚡ 초고속 ({duration*1000:.0f}ms)"
        elif duration < 5:
            return f"🚀 빠름 ({duration:.2f}s)"
        elif duration < 15:
            return f"⏱️ 보통 ({duration:.2f}s)"
        else:
            return f"🐌 느림 ({duration:.2f}s) - 최적화 필요"
    
    def print_total_summary(self):
        """전체 성능 요약 출력"""
        total_time = time.time() - self.start_time
        
        print(f"\n" + "="*100)
        print(f"📊 전체 성능 분석 보고서")
        print(f"="*100)
        print(f"🕐 총 실행 시간: {total_time:.3f}초")
        
        print(f"\n📋 단계별 상세 시간 분석:")
        print(f"{'단계명':<25} {'시작시간':<12} {'종료시간':<12} {'소요시간':<10} {'성능평가':<15}")
        print("-" * 90)
        
        for stage_name, times in self.stage_times.items():
            start_time = times.get("start_datetime", "N/A")
            end_time = times.get("end_datetime", "N/A")
            duration = times.get("duration", 0)
            performance = self.get_performance_summary(stage_name)
            
            print(f"{stage_name:<25} {start_time:<12} {end_time:<12} {duration:<9.3f}s {performance:<15}")
        
        print(f"\n🎯 병목점 분석:")
        sorted_stages = sorted(self.stage_times.items(), 
                              key=lambda x: x[1].get("duration", 0), reverse=True)
        
        for i, (stage_name, times) in enumerate(sorted_stages[:3], 1):
            duration = times.get("duration", 0)
            percentage = (duration / total_time) * 100
            print(f"   {i}. {stage_name}: {duration:.3f}초 ({percentage:.1f}%)")
        
        print(f"\n💡 최적화 제안:")
        for stage_name, times in sorted_stages:
            duration = times.get("duration", 0)
            if duration > 5:
                print(f"   ⚠️ {stage_name}: {duration:.2f}초로 최적화 필요")
            elif duration > 1:
                print(f"   🔍 {stage_name}: {duration:.2f}초로 개선 여지 있음")
        
        return total_time

monitor = DetailedPerformanceMonitor()

# 1단계: 고성능 병렬 문서 로딩
def load_single_file(file_path):
    """단일 파일 로딩 함수 (최적화)"""
    try:
        # 파일 크기 확인 후 적절한 버퍼 크기 설정
        file_size = os.path.getsize(file_path)
        buffer_size = min(8192 * 16, file_size)  # 128KB 버퍼 또는 파일 크기
        
        with open(file_path, 'r', encoding='utf-8', buffering=buffer_size) as f:
            content = f.read()
        return {
            "page_content": content,
            "metadata": {"source": str(file_path), "size": file_size}
        }
    except Exception as e:
        print(f"   ❌ 파일 로딩 실패: {file_path} - {str(e)}")
        return None

def high_performance_document_loading():
    """16코어 최적화 병렬 문서 로딩"""
    stage_name = "고성능_문서_로딩"
    monitor.log_stage_start(stage_name)
    
    data_dir = Path("./data")
    md_files = list(data_dir.glob("*.md"))
    
    if not md_files:
        print(f"   ⚠️ ./data 폴더에 .md 파일이 없습니다!")
        duration = monitor.log_stage_end(stage_name)
        return []
    
    docs = []
    
    # 16코어 시스템 최적화: 32개 워커
    with ThreadPoolExecutor(max_workers=PARALLELISM["file_loading"]) as executor:
        future_to_file = {executor.submit(load_single_file, file_path): file_path 
                         for file_path in md_files}
        
        with tqdm(total=len(md_files), desc="📄 고성능 파일 로딩", 
                 bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]") as pbar:
            
            for future in as_completed(future_to_file):
                file_path = future_to_file[future]
                try:
                    doc = future.result()
                    if doc:
                        docs.append(doc)
                        file_size = doc["metadata"]["size"]
                        pbar.set_description(f"📄 로딩: {file_path.name} ({file_size//1024}KB)")
                    pbar.update(1)
                except Exception as e:
                    pbar.set_description(f"❌ 실패: {file_path.name}")
                    pbar.update(1)
    
    duration = monitor.log_stage_end(stage_name)
    
    if docs:
        total_size = sum(doc["metadata"]["size"] for doc in docs) / (1024*1024)
        throughput = total_size / duration if duration > 0 else 0
        print(f"   📈 처리량: {total_size:.2f}MB, {throughput:.2f}MB/s")
    
    return docs

# 2단계: 고성능 스레드 기반 텍스트 분할 (Jupyter 호환)
def split_documents_threaded(docs):
    """스레드 기반 텍스트 분할 (Jupyter 환경 호환)"""
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    
    # 16코어 시스템 최적화: 더 큰 청크 크기
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,      # 300 -> 500으로 증가
        chunk_overlap=100,   # 50 -> 100으로 증가
        separators=["\n\n", "\n", ".", " ", ""]
    )
    
    all_split_docs = []
    
    for doc_data in docs:
        try:
            # 문서 객체 재구성
            class Document:
                def __init__(self, page_content, metadata):
                    self.page_content = page_content
                    self.metadata = metadata
            
            doc = Document(doc_data["page_content"], doc_data["metadata"])
            chunks = text_splitter.split_documents([doc])
            all_split_docs.extend(chunks)
        except Exception as e:
            print(f"   ❌ 문서 분할 실패: {e}")
            continue
    
    return all_split_docs

def high_performance_text_splitting(docs):
    """16코어 최적화 스레드 기반 텍스트 분할"""
    stage_name = "고성능_텍스트_분할"
    monitor.log_stage_start(stage_name)
    
    if not docs:
        print(f"   ⚠️ 분할할 문서가 없습니다!")
        duration = monitor.log_stage_end(stage_name)
        return []
    
    split_docs = []
    
    # ThreadPoolExecutor로 안전한 병렬 처리 (Jupyter 호환)
    with ThreadPoolExecutor(max_workers=PARALLELISM["text_splitting"]) as executor:
        # 문서를 청크로 나누어 병렬 처리
        chunk_size = max(1, len(docs) // PARALLELISM["text_splitting"])
        doc_chunks = [docs[i:i + chunk_size] for i in range(0, len(docs), chunk_size)]
        
        future_to_chunk = {executor.submit(split_documents_threaded, chunk): i 
                          for i, chunk in enumerate(doc_chunks)}
        
        with tqdm(total=len(doc_chunks), desc="✂️ 고성능 텍스트 분할",
                 bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]") as pbar:
            
            for future in as_completed(future_to_chunk):
                chunk_idx = future_to_chunk[future]
                try:
                    chunks = future.result()
                    split_docs.extend(chunks)
                    pbar.set_description(f"✂️ 청크{chunk_idx+1} -> {len(chunks)}개 분할")
                    pbar.update(1)
                except Exception as e:
                    pbar.set_description(f"❌ 청크{chunk_idx+1} 분할 실패")
                    pbar.update(1)
    
    duration = monitor.log_stage_end(stage_name)
    
    chunk_rate = len(split_docs) / duration if duration > 0 else 0
    print(f"   📈 분할 성능: {len(split_docs)}개 청크, {chunk_rate:.1f}청크/초")
    
    return split_docs

# 3단계: 64개 동시 요청 임베딩 최적화
def setup_ultra_optimized_embeddings():
    """64개 동시 요청 임베딩 설정"""
    stage_name = "초고성능_임베딩_설정"
    monitor.log_stage_start(stage_name)
    
    from langchain_openai import OpenAIEmbeddings
    
    # 16코어 시스템: 64개 동시 요청
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small",
        max_retries=5,
        request_timeout=120,
        chunk_size=min(2000, PARALLELISM["embedding"]),  # 2000개 또는 64개
    )
    
    duration = monitor.log_stage_end(stage_name)
    print(f"   ⚡ 임베딩 병렬도: {PARALLELISM['embedding']}개 동시 요청")
    
    return embeddings

# 4단계: 초고속 BM25 인덱싱
def ultra_fast_bm25_setup(split_docs):
    """128GB 메모리 활용 초고속 BM25 인덱싱"""
    stage_name = "초고속_BM25_인덱싱"
    monitor.log_stage_start(stage_name)
    
    if not split_docs:
        print(f"   ⚠️ 인덱싱할 문서가 없습니다!")
        duration = monitor.log_stage_end(stage_name)
        return None
    
    bm25_cache_path = os.path.join(CACHE_DIR, "bm25_retriever_jupyter_v4.pkl")
    
    # 캐시 확인
    if os.path.exists(bm25_cache_path):
        cache_start = time.time()
        with tqdm(total=1, desc="📋 BM25 캐시 로딩") as pbar:
            with open(bm25_cache_path, 'rb') as f:
                bm25_retriever = pickle.load(f)
            pbar.update(1)
        
        cache_time = time.time() - cache_start
        duration = monitor.log_stage_end(stage_name)
        print(f"   ⚡ 캐시 로딩 속도: {cache_time:.3f}초")
        return bm25_retriever
    
    # 새로 생성 - 128GB 메모리 활용
    from langchain_community.retrievers import BM25Retriever
    
    # 메모리 풍부한 시스템: 한 번에 모든 문서 처리
    index_start = time.time()
    
    with tqdm(total=len(split_docs), desc="🔍 초고속 BM25 인덱싱",
             bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]") as pbar:
        
        try:
            # 128GB 메모리: 모든 문서를 메모리에 로드하여 처리
            bm25_retriever = BM25Retriever.from_documents(split_docs)
            bm25_retriever.k = 5
            
            pbar.update(len(split_docs))
        except Exception as e:
            print(f"   ❌ BM25 인덱싱 실패: {e}")
            duration = monitor.log_stage_end(stage_name)
            return None
    
    index_time = time.time() - index_start
    
    # 캐시 저장
    cache_save_start = time.time()
    with open(bm25_cache_path, 'wb') as f:
        pickle.dump(bm25_retriever, f)
    cache_save_time = time.time() - cache_save_start
    
    duration = monitor.log_stage_end(stage_name)
    
    indexing_rate = len(split_docs) / index_time if index_time > 0 else 0
    print(f"   📈 인덱싱 성능: {indexing_rate:.1f}문서/초")
    print(f"   💾 캐시 저장: {cache_save_time:.3f}초")
    
    return bm25_retriever

# 5단계: 전체 검색기 초고속 설정
def setup_ultra_high_performance_retrievers():
    """16코어 128GB 시스템 최적화 검색기 설정"""
    stage_name = "전체_검색기_초고속_설정"
    monitor.log_stage_start(stage_name)
    
    # 전체 진행률 표시
    main_progress = tqdm(total=6, desc="🏗️ 초고성능 검색기 시스템 구축", position=0,
                        bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]")
    
    # 1. 고성능 문서 로딩
    main_progress.set_description("📄 고성능 문서 로딩...")
    docs = high_performance_document_loading()
    main_progress.update(1)
    
    # 2. 고성능 텍스트 분할
    main_progress.set_description("✂️ 고성능 텍스트 분할...")
    split_docs = high_performance_text_splitting(docs)
    main_progress.update(1)
    
    # 3. 초고성능 임베딩 설정
    main_progress.set_description("🧠 초고성능 임베딩 설정...")
    embeddings = setup_ultra_optimized_embeddings()
    main_progress.update(1)
    
    # 4. 의미 검색기 설정
    main_progress.set_description("🧠 의미 검색기 최적화...")
    try:
        semantic_retriever = chroma_db.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5}
        )
    except Exception as e:
        print(f"   ❌ ChromaDB 검색기 설정 실패: {e}")
        semantic_retriever = None
    main_progress.update(1)
    
    # 5. 초고속 BM25 검색기
    main_progress.set_description("🔍 초고속 BM25 인덱싱...")
    bm25_retriever = ultra_fast_bm25_setup(split_docs)
    main_progress.update(1)
    
    # 6. 고급 검색기 구성
    main_progress.set_description("🔄 고급 검색기 최적화...")
    
    retrievers = {}
    
    if semantic_retriever:
        retrievers["semantic"] = semantic_retriever
    
    if bm25_retriever:
        retrievers["bm25"] = bm25_retriever
        
        # 하이브리드 검색기 (의미 검색기가 있을 때만)
        if semantic_retriever:
            from langchain.retrievers import EnsembleRetriever
            hybrid_retriever = EnsembleRetriever(
                retrievers=[semantic_retriever, bm25_retriever],
                weights=[0.7, 0.3]  # 의미 검색 비중 증가
            )
            retrievers["hybrid"] = hybrid_retriever
            
            # 멀티쿼리 검색기
            try:
                from langchain.retrievers.multi_query import MultiQueryRetriever
                from langchain_openai import ChatOpenAI
                
                llm = ChatOpenAI(
                    model="gpt-4o-mini", 
                    temperature=0,
                    request_timeout=60,
                    max_retries=3
                )
                
                multi_query_retriever = MultiQueryRetriever.from_llm(
                    retriever=hybrid_retriever,
                    llm=llm
                )
                retrievers["multi_query"] = multi_query_retriever
            except Exception as e:
                print(f"   ❌ 멀티쿼리 검색기 설정 실패: {e}")
    
    main_progress.update(1)
    main_progress.close()
    
    duration = monitor.log_stage_end(stage_name)
    
    print(f"   🎯 생성된 검색기: {len(retrievers)}개")
    print(f"   📊 처리된 문서: {len(docs)}개 -> {len(split_docs)}개 청크")
    
    return retrievers

# 6단계: 8개 동시 초고속 검색 테스트
def ultra_high_speed_search_test(query, retrievers_dict):
    """8개 동시 초고속 검색 테스트"""
    stage_name = "초고속_병렬_검색_테스트"
    monitor.log_stage_start(stage_name)
    
    if not retrievers_dict:
        print(f"   ⚠️ 검색할 검색기가 없습니다!")
        duration = monitor.log_stage_end(stage_name)
        return {}, duration
    
    results = {}
    
    # 시스템 상태 출력
    initial_stats = monitor.get_detailed_system_stats()
    print(f"   🖥️ 검색 시작 시점 시스템 상태: {initial_stats}")
    
    with ThreadPoolExecutor(max_workers=PARALLELISM["search_test"]) as executor:
        future_to_name = {}
        
        # 모든 검색기에 대해 Future 생성
        for name, retriever in retrievers_dict.items():
            future = executor.submit(retriever.invoke, query)
            future_to_name[future] = name
        
        # 실시간 성능 모니터링
        with tqdm(total=len(future_to_name), desc="🚀 초고속 병렬 검색",
                 bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]") as pbar:
            
            for future in as_completed(future_to_name):
                name = future_to_name[future]
                search_start = time.time()
                
                try:
                    docs = future.result(timeout=60)
                    search_time = time.time() - search_start
                    results[name] = docs[:3]
                    
                    # 개별 검색기 성능 측정
                    current_stats = monitor.get_detailed_system_stats()
                    cpu_info = current_stats.get('cpu_avg', 'N/A')
                    pbar.set_description(f"✅ {name} 완료 ({search_time:.3f}초, CPU:{cpu_info})")
                    pbar.update(1)
                    
                except Exception as e:
                    search_time = time.time() - search_start
                    results[name] = []
                    pbar.set_description(f"❌ {name} 실패 ({search_time:.3f}초)")
                    pbar.update(1)
    
    duration = monitor.log_stage_end(stage_name)
    
    search_rate = len(retrievers_dict) / duration if duration > 0 else 0
    print(f"   ⚡ 검색 성능: {search_rate:.1f}검색기/초")
    
    return results, duration

# 메인 실행
print("\n" + "="*100)
print("🚀 16코어 128GB 시스템 최적화 초고성능 검색기 실행 (Jupyter 호환)")
print("="*100)

total_start = time.time()

# 모든 검색기 설정
retrievers = setup_ultra_high_performance_retrievers()

if not retrievers:
    print("❌ 검색기 설정에 실패했습니다. 데이터 폴더와 ChromaDB를 확인해주세요.")
else:
    # 테스트 쿼리
    test_query = "테슬라 트럭 모델이 있나요?"
    print(f"\n📋 테스트 쿼리: '{test_query}'")

    # 초고속 병렬 검색 실행
    search_results, search_time = ultra_high_speed_search_test(test_query, retrievers)

    # 전체 성능 분석 출력
    total_time = monitor.print_total_summary()

    # 검색 결과 출력
    print(f"\n" + "="*100)
    print(f"🎯 초고성능 병렬 검색 결과")
    print("="*100)

    result_icons = {"semantic": "🧠", "bm25": "🔍", "hybrid": "🔄", "multi_query": "🎯"}
    result_names = {"semantic": "의미 검색", "bm25": "키워드 검색", 
                   "hybrid": "하이브리드 검색", "multi_query": "멀티쿼리 검색"}

    for name, docs in search_results.items():
        print(f"\n{result_icons.get(name, '🔧')} === {result_names.get(name, name)} 결과 ===")
        if docs:
            for i, doc in enumerate(docs, 1):
                source = doc.metadata.get('source', 'Unknown')
                print(f"{i}. {doc.page_content[:80]}... [출처: {source}]")
        else:
            print("❌ 검색 결과 없음 또는 오류 발생")

    print(f"\n" + "="*100)
    print("🎉 16코어 128GB 최적화 실습 1 완료! (Jupyter 호환)")
    print("📊 최대 성능 활용:")
    print(f"   🔥 파일 로딩: {PARALLELISM['file_loading']}개 스레드")
    print(f"   ⚡ 텍스트 분할: {PARALLELISM['text_splitting']}개 스레드 (Jupyter 안전 모드)")
    print(f"   🚀 임베딩 요청: {PARALLELISM['embedding']}개 동시")
    print(f"   💾 BM25 인덱싱: {PARALLELISM['bm25_indexing']}개 스레드")
    print(f"   🎯 검색 테스트: {PARALLELISM['search_test']}개 동시")
    print("   ⏰ 모든 단계별 상세 시간 측정")
    print("   📈 실시간 성능 모니터링")
    print("   🔍 병목점 자동 분석")
    print("   ✅ Jupyter 환경 호환성 확보")
    print("="*100)

🚀 16코어 128GB 최적화 RAG 검색기 초기화 시작!
💻 시스템 정보: CPU 16코어, 메모리 127GB
🔥 Jupyter 최적화 병렬도: {'file_loading': 32, 'text_splitting': 24, 'embedding': 64, 'bm25_indexing': 24, 'search_test': 8}

🚀 16코어 128GB 시스템 최적화 초고성능 검색기 실행 (Jupyter 호환)

⏰ [12:51:37.591] 🚀 전체_검색기_초고속_설정 시작
   💻 시작 시 시스템 상태: {'cpu_avg': '2.1%', 'cpu_cores': ['C0:0.0%', 'C1:0.0%', 'C2:16.7%', 'C3:0.0%', 'C4:0.0%', 'C5:0.0%', 'C6:0.0%', 'C7:0.0%'], 'memory_used': '35GB/127GB (27.4%)', 'memory_available': '92GB', 'active_threads': 16}


📄 고성능 문서 로딩...:   0%|          | 0/6 [00:00<?]        


⏰ [12:51:37.701] 🚀 고성능_문서_로딩 시작
   💻 시작 시 시스템 상태: {'cpu_avg': '10.4%', 'cpu_cores': ['C0:50.0%', 'C1:0.0%', 'C2:33.3%', 'C3:0.0%', 'C4:0.0%', 'C5:0.0%', 'C6:16.7%', 'C7:0.0%'], 'memory_used': '35GB/127GB (27.4%)', 'memory_available': '92GB', 'active_threads': 16}


📄 로딩: Tesla_EN.md (12KB): 100%|██████████| 4/4 [00:00<00:00, 799.68it/s]


⏰ [12:51:37.821] ✅ 고성능_문서_로딩 완료
   ⏱️  소요시간: 0.120초


...

⏰ [12:51:39.003] ✅ 초고속_BM25_인덱싱 완료
   ⏱️  소요시간: 0.120초
   💻 종료 시 시스템 상태: {'cpu_avg': '13.8%', 'cpu_cores': ['C0:16.7%', 'C1:0.0%', 'C2:16.7%', 'C3:0.0%', 'C4:0.0%', 'C5:0.0%', 'C6:42.9%', 'C7:0.0%'], 'memory_used': '35GB/127GB (27.5%)', 'memory_available': '92GB', 'active_threads': 16}
   📊 성능 요약: ⚡ 초고속 (120ms)
   📈 인덱싱 성능: 8898.3문서/초
   💾 캐시 저장: 0.002초


🔄 고급 검색기 최적화...: 100%|██████████| 6/6 [00:01<00:00]


⏰ [12:51:39.618] ✅ 전체_검색기_초고속_설정 완료
   ⏱️  소요시간: 2.027초
   💻 종료 시 시스템 상태: {'cpu_avg': '3.1%', 'cpu_cores': ['C0:0.0%', 'C1:0.0%', 'C2:0.0%', 'C3:0.0%', 'C4:0.0%', 'C5:0.0%', 'C6:0.0%', 'C7:0.0%'], 'memory_used': '35GB/127GB (27.4%)', 'memory_available': '92GB', 'active_threads': 16}
   📊 성능 요약: 🚀 빠름 (2.03s)
   🎯 생성된 검색기: 4개
   📊 처리된 문서: 4개 -> 89개 청크

📋 테스트 쿼리: '테슬라 트럭 모델이 있나요?'

⏰ [12:51:39.725] 🚀 초고속_병렬_검색_테스트 시작
   💻 시작 시 시스템 상태: {'cpu_avg': '7.3%', 'cpu_cores': ['C0:16.7%', 'C1:0.0%', 'C2:33.3%', 'C3:0.0%', 'C4:0.0%', 'C5:0.0%', 'C6:16.7%', 'C7:0.0%'], 'memory_used': '35GB/127GB (27.4%)', 'memory_available': '92GB', 'active_threads': 16}
   🖥️ 검색 시작 시점 시스템 상태: {'cpu_avg': '5.4%', 'cpu_cores': ['C0:0.0%', 'C1:0.0%', 'C2:14.3%', 'C3:0.0%', 'C4:0.0%', 'C5:0.0%', 'C6:28.6%', 'C7:0.0%'], 'memory_used': '35GB/127GB (27.4%)', 'memory_available': '92GB', 'active_threads': 16}


✅ multi_query 완료 (0.000초, CPU:3.4%): 100%|██████████| 4/4 [00:03<00:00,  1.33it/s]

⏰ [12:51:42.955] ✅ 초고속_병렬_검색_테스트 완료
   ⏱️  소요시간: 3.230초
   💻 종료 시 시스템 상태: {'cpu_avg': '14.4%', 'cpu_cores': ['C0:44.4%', 'C1:0.0%', 'C2:28.6%', 'C3:0.0%', 'C4:0.0%', 'C5:0.0%', 'C6:28.6%', 'C7:0.0%'], 'memory_used': '35GB/127GB (27.5%)', 'memory_available': '92GB', 'active_threads': 16}
   📊 성능 요약: 🚀 빠름 (3.23s)
   ⚡ 검색 성능: 1.2검색기/초

📊 전체 성능 분석 보고서
🕐 총 실행 시간: 5.475초

📋 단계별 상세 시간 분석:
단계명                       시작시간         종료시간         소요시간       성능평가           
------------------------------------------------------------------------------------------
전체_검색기_초고속_설정             12:51:37.591 12:51:39.618 2.027    s 🚀 빠름 (2.03s)   
고성능_문서_로딩                 12:51:37.701 12:51:37.821 0.120    s ⚡ 초고속 (120ms)  
고성능_텍스트_분할                12:51:37.931 12:51:38.052 0.121    s ⚡ 초고속 (121ms)  
초고성능_임베딩_설정               12:51:38.162 12:51:38.772 0.611    s ⚡ 초고속 (611ms)  
초고속_BM25_인덱싱              12:51:38.883 12:51:39.003 0.120    s ⚡ 초고속 (120ms)  
초고속_병렬_검색_테스트             12:51:39.725 12:51:42.95

`(2) 검색기법 고도화`

- Rerank, Comporession 기법을 적용합니다. 
- Pipeline Compressor로 연결하여 구성합니다. 

In [22]:
# 실습 2: 검색기법 고도화 (Rerank + Compression)

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import (
    LLMListwiseRerank, 
    LLMChainExtractor,
    EmbeddingsFilter,
    DocumentCompressorPipeline
)
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_openai import ChatOpenAI

print("🔧 실습 2: 검색기법 고도화 시작")

# 실습 1에서 생성된 검색기들 확인 및 재생성 (필요시)
try:
    # 실습 1의 변수들이 있는지 확인
    test_vars = [
        ('semantic_retriever', 'semantic_retriever'),
        ('bm25_retriever', 'bm25_retriever'), 
        ('hybrid_retriever', 'hybrid_retriever'),
        ('retrievers', 'retrievers')
    ]
    
    missing_vars = []
    for var_name, display_name in test_vars:
        try:
            eval(var_name)
            print(f"✅ {display_name} 발견됨")
        except NameError:
            missing_vars.append(var_name)
            print(f"❌ {display_name} 없음")
    
    if missing_vars:
        print(f"\n⚠️ 실습 1의 일부 변수가 없습니다: {missing_vars}")
        print("🔄 필요한 검색기들을 재생성합니다...")
        
        # 기본 검색기들 재생성
        print("\n📋 검색기 재생성 중...")
        
        # 1. Semantic retriever (ChromaDB 기반)
        try:
            semantic_retriever = chroma_db.as_retriever(
                search_type="similarity",
                search_kwargs={"k": 5}
            )
            print("✅ semantic_retriever 생성 완료")
        except Exception as e:
            print(f"❌ semantic_retriever 생성 실패: {e}")
            semantic_retriever = None
        
        # 2. BM25 retriever 캐시에서 로드 또는 재생성
        try:
            import os
            import pickle
            from pathlib import Path
            
            bm25_cache_path = "./cache/bm25_retriever_jupyter_v4.pkl"
            
            if os.path.exists(bm25_cache_path):
                print("📋 BM25 캐시에서 로드 중...")
                with open(bm25_cache_path, 'rb') as f:
                    bm25_retriever = pickle.load(f)
                print("✅ bm25_retriever 캐시 로드 완료")
            else:
                print("🔄 BM25 retriever 새로 생성 중...")
                
                # 문서 로드 및 분할
                from langchain_community.document_loaders import DirectoryLoader
                from langchain.text_splitter import RecursiveCharacterTextSplitter
                from langchain_community.retrievers import BM25Retriever
                
                data_dir = Path("./data")
                md_files = list(data_dir.glob("*.md"))
                
                if md_files:
                    # 문서 로드
                    docs = []
                    for file_path in md_files:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            content = f.read()
                        docs.append({
                            "page_content": content,
                            "metadata": {"source": str(file_path)}
                        })
                    
                    # 텍스트 분할
                    text_splitter = RecursiveCharacterTextSplitter(
                        chunk_size=500,
                        chunk_overlap=100,
                        separators=["\n\n", "\n", ".", " ", ""]
                    )
                    
                    class Document:
                        def __init__(self, page_content, metadata):
                            self.page_content = page_content
                            self.metadata = metadata
                    
                    split_docs = []
                    for doc_data in docs:
                        doc = Document(doc_data["page_content"], doc_data["metadata"])
                        chunks = text_splitter.split_documents([doc])
                        split_docs.extend(chunks)
                    
                    # BM25 생성
                    bm25_retriever = BM25Retriever.from_documents(split_docs)
                    bm25_retriever.k = 5
                    
                    # 캐시 저장
                    os.makedirs("./cache", exist_ok=True)
                    with open(bm25_cache_path, 'wb') as f:
                        pickle.dump(bm25_retriever, f)
                    
                    print("✅ bm25_retriever 생성 및 캐시 저장 완료")
                else:
                    print("❌ ./data 폴더에 .md 파일이 없습니다!")
                    bm25_retriever = None
                    
        except Exception as e:
            print(f"❌ bm25_retriever 생성 실패: {e}")
            bm25_retriever = None
        
        # 3. Hybrid retriever 생성
        if semantic_retriever and bm25_retriever:
            try:
                from langchain.retrievers import EnsembleRetriever
                
                hybrid_retriever = EnsembleRetriever(
                    retrievers=[semantic_retriever, bm25_retriever],
                    weights=[0.7, 0.3]  # semantic: 70%, keyword: 30%
                )
                print("✅ hybrid_retriever 생성 완료")
            except Exception as e:
                print(f"❌ hybrid_retriever 생성 실패: {e}")
                hybrid_retriever = semantic_retriever  # fallback
        elif semantic_retriever:
            hybrid_retriever = semantic_retriever
            print("⚠️ semantic_retriever를 hybrid_retriever로 사용")
        elif bm25_retriever:
            hybrid_retriever = bm25_retriever
            print("⚠️ bm25_retriever를 hybrid_retriever로 사용")
        else:
            print("❌ 검색기 생성에 완전히 실패했습니다!")
            hybrid_retriever = None
    
    else:
        print("✅ 모든 필수 검색기가 준비되었습니다!")
        
        # 실습 1의 retrievers 딕셔너리에서 가져오기
        if 'retrievers' in locals() or 'retrievers' in globals():
            retrievers_dict = eval('retrievers')
            if 'hybrid' in retrievers_dict:
                hybrid_retriever = retrievers_dict['hybrid']
                print("✅ 실습 1의 hybrid_retriever 사용")
            elif 'semantic' in retrievers_dict:
                hybrid_retriever = retrievers_dict['semantic']
                print("⚠️ 실습 1의 semantic_retriever를 hybrid_retriever로 사용")
        else:
            # 변수가 직접 정의되어 있는 경우
            if 'hybrid_retriever' not in locals() and 'hybrid_retriever' not in globals():
                hybrid_retriever = semantic_retriever
                print("⚠️ semantic_retriever를 hybrid_retriever로 사용")

except Exception as e:
    print(f"❌ 검색기 확인 중 오류: {e}")
    # 최소한의 검색기라도 설정
    try:
        hybrid_retriever = chroma_db.as_retriever(search_kwargs={"k": 5})
        print("✅ ChromaDB를 기본 검색기로 사용")
    except:
        print("❌ 모든 검색기 설정 실패")
        hybrid_retriever = None

# 검색기가 준비되었는지 최종 확인
if hybrid_retriever is None:
    print("❌ 실습 2를 진행할 수 없습니다. 검색기가 없습니다!")
    print("💡 실습 1을 먼저 실행해주세요.")
else:
    print(f"✅ 실습 2 준비 완료! 사용 검색기: {type(hybrid_retriever).__name__}")

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("\n🔧 고도화 기법 구성 시작...")

# 1) LLM Reranker 설정
print("1️⃣ LLM Reranker 설정...")
llm_reranker = LLMListwiseRerank.from_llm(llm, top_n=3)

# 2) LLM Chain Extractor 설정 (맥락 압축)
print("2️⃣ LLM Chain Extractor 설정...")
llm_extractor = LLMChainExtractor.from_llm(llm)

# 3) Embeddings Filter 설정 (유사도 기반 필터링)
print("3️⃣ Embeddings Filter 설정...")
embeddings_filter = EmbeddingsFilter(
    embeddings=embeddings, 
    similarity_threshold=0.5
)

# 4) Redundant Filter 설정 (중복 제거)
print("4️⃣ Redundant Filter 설정...")
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

# 5) Pipeline Compressor 구성
print("5️⃣ Pipeline Compressor 구성...")
# 순서: 중복제거 -> 유사도 필터링 -> 맥락 압축 -> 재순위화
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[
        redundant_filter,      # 1단계: 중복 문서 제거
        embeddings_filter,     # 2단계: 유사도 기반 필터링
        llm_extractor,         # 3단계: 관련 내용만 추출
        llm_reranker          # 4단계: 최종 재순위화
    ]
)

# 6) 최종 고도화된 검색기 구성
if hybrid_retriever is not None:
    print("6️⃣ 고도화된 검색기 구성...")
    advanced_retriever = ContextualCompressionRetriever(
        base_compressor=pipeline_compressor,
        base_retriever=hybrid_retriever  # 준비된 하이브리드 검색기 사용
    )
    
    # 테스트 및 결과 비교
    test_queries = [
        "테슬라 트럭 모델이 있나요?",
        "리비안의 전기 트럭 특징은?",
        "테슬라와 리비안의 차이점은?"
    ]

    print(f"\n🎯 고도화 검색 테스트 시작 (총 {len(test_queries)}개 쿼리)")

    for i, query in enumerate(test_queries, 1):
        print(f"\n{'='*80}")
        print(f"🔍 쿼리 {i}: {query}")
        print(f"{'='*80}")
        
        try:
            # 기본 검색 결과
            print("\n--- 기본 검색 결과 ---")
            basic_docs = hybrid_retriever.invoke(query)
            print(f"검색된 문서 수: {len(basic_docs)}")
            for j, doc in enumerate(basic_docs[:2], 1):
                print(f"{j}. {doc.page_content[:150]}...")
            
            # 고도화된 검색 결과
            print("\n--- 고도화된 검색 결과 (Pipeline) ---")
            advanced_docs = advanced_retriever.invoke(query, config={"callbacks": [langfuse_handler]})
            print(f"최종 문서 수: {len(advanced_docs)}")
            for j, doc in enumerate(advanced_docs, 1):
                print(f"{j}. {doc.page_content}")
                print(f"   [출처: {doc.metadata['source']}]")
                print()
        
        except Exception as e:
            print(f"❌ 쿼리 '{query}' 처리 중 오류: {e}")
            continue

    print(f"\n🎯 Pipeline 구성:")
    print("1. 중복 제거 (EmbeddingsRedundantFilter)")
    print("2. 유사도 필터링 (EmbeddingsFilter, threshold=0.5)")  
    print("3. 맥락 압축 (LLMChainExtractor)")
    print("4. 재순위화 (LLMListwiseRerank, top_n=3)")
    
    print(f"\n✅ 실습 2 완료!")
    print(f"📊 사용된 검색기: {type(hybrid_retriever).__name__}")
    print(f"🔧 적용된 기법: 4단계 파이프라인 압축 + 재순위화")

else:
    print("❌ 실습 2를 완료할 수 없습니다. 기본 검색기가 필요합니다.")
    print("💡 실습 1을 먼저 성공적으로 실행해주세요.")

🔧 실습 2: 검색기법 고도화 시작
❌ semantic_retriever 없음
❌ bm25_retriever 없음
❌ hybrid_retriever 없음
✅ retrievers 발견됨

⚠️ 실습 1의 일부 변수가 없습니다: ['semantic_retriever', 'bm25_retriever', 'hybrid_retriever']
🔄 필요한 검색기들을 재생성합니다...

📋 검색기 재생성 중...
✅ semantic_retriever 생성 완료
📋 BM25 캐시에서 로드 중...
✅ bm25_retriever 캐시 로드 완료
✅ hybrid_retriever 생성 완료
✅ 실습 2 준비 완료! 사용 검색기: EnsembleRetriever

🔧 고도화 기법 구성 시작...
1️⃣ LLM Reranker 설정...
2️⃣ LLM Chain Extractor 설정...
3️⃣ Embeddings Filter 설정...
4️⃣ Redundant Filter 설정...
5️⃣ Pipeline Compressor 구성...
6️⃣ 고도화된 검색기 구성...

🎯 고도화 검색 테스트 시작 (총 3개 쿼리)

🔍 쿼리 1: 테슬라 트럭 모델이 있나요?

--- 기본 검색 결과 ---
검색된 문서 수: 10
1. [출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
- **Cybertruck:** 2019년 11월에 처음 발표된 풀사이즈 픽업 트럭. 후륜 구동, 듀얼 모터 전륜 구동, 트리 모터 전륜 구동의 세 가지 모델이...
2. [출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
| Tesla 모델 |          |       |                  |
| :--------- | :------- | :---- | :---...

--- 고도화된 검색 결과 (Pipeline) ---
최종 문서 수: 2
1. - **Cybertruck:** 2

`(3) RAG 체인 연결`

- 검색기, 프롬프트, LLM을 LCEL로 연결하여 RAG Chain을 구성합니다. 
- 다양한 쿼리를 입력하고, 생성된 답변의 품질을 평가합니다. 

In [23]:
# 실습 3: RAG 체인 연결 (LCEL 사용)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI

# 1) 프롬프트 템플릿 정의
rag_prompt = ChatPromptTemplate.from_template("""
당신은 전기차 전문가입니다. 주어진 문서를 바탕으로 정확하고 도움이 되는 답변을 제공해주세요.

검색된 문서:
{context}

질문: {question}

답변 가이드라인:
1. 문서에 있는 정보만을 사용하여 답변하세요
2. 정확한 모델명, 연도, 사양 등을 포함하세요  
3. 출처를 명시하세요
4. 문서에 정보가 없다면 "제공된 문서에서는 해당 정보를 찾을 수 없습니다"라고 답변하세요

답변:
""")

# 2) LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# 3) 문서 포맷팅 함수
def format_docs(docs):
    formatted_docs = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        content = doc.page_content.strip()
        formatted_docs.append(f"문서 {i} [출처: {source}]:\n{content}")
    return "\n\n" + "\n\n".join(formatted_docs)

# 4) RAG 체인 구성 (LCEL)
rag_chain = (
    RunnableParallel({
        "context": advanced_retriever | format_docs,  # 실습2의 고도화된 검색기 사용
        "question": RunnablePassthrough()
    })
    | rag_prompt
    | llm
    | StrOutputParser()
)

# 5) 다양한 쿼리로 성능 테스트
test_questions = [
    "테슬라 Cybertruck의 특징과 출시년도를 알려주세요",
    "리비안 R1T와 테슬라 Cybertruck의 차이점은 무엇인가요?",
    "테슬라에서 생산하는 트럭 모델들을 모두 나열해주세요",
    "전기 픽업트럭 중에서 어떤 모델을 추천하시나요?",
    "테슬라 Model S의 제조년도와 좌석 수는?",
    "2025년에 출시 예정인 테슬라 모델이 있나요?"
]

print("🚗 RAG 체인 성능 테스트")
print("="*80)

for i, question in enumerate(test_questions, 1):
    print(f"\n질문 {i}: {question}")
    print("-" * 60)
    
    try:
        # RAG 체인 실행
        answer = rag_chain.invoke(question, config={"callbacks": [langfuse_handler]})
        print(f"답변: {answer}")
        
        # 검색된 문서 수 확인
        retrieved_docs = advanced_retriever.invoke(question)
        print(f"📊 검색된 문서 수: {len(retrieved_docs)}")
        
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
    
    print("\n" + "="*80)

# 6) 성능 평가 메트릭
print("\n📈 RAG 시스템 구성 요약:")
print("├── 검색기: Hybrid (Semantic + BM25) + MultiQuery")
print("├── 고도화: Pipeline (중복제거 → 유사도필터 → 압축 → 재순위)")
print("├── 프롬프트: 전기차 전문가 역할 + 명확한 가이드라인")
print("├── LLM: GPT-4o-mini (temperature=0.1)")
print("└── 추적: Langfuse 콜백으로 성능 모니터링")

# 7) 간단한 성능 비교 (기본 vs 고도화)
print("\n🔍 성능 비교 테스트:")
comparison_query = "테슬라 트럭 모델이 있나요?"

# 기본 검색기로 RAG
basic_rag_chain = (
    RunnableParallel({
        "context": semantic_retriever | format_docs,
        "question": RunnablePassthrough()
    })
    | rag_prompt
    | llm
    | StrOutputParser()
)

print(f"\n질문: {comparison_query}")
print("\n--- 기본 RAG 결과 ---")
basic_answer = basic_rag_chain.invoke(comparison_query)
print(basic_answer)

print("\n--- 고도화된 RAG 결과 ---")
advanced_answer = rag_chain.invoke(comparison_query)
print(advanced_answer)

🚗 RAG 체인 성능 테스트

질문 1: 테슬라 Cybertruck의 특징과 출시년도를 알려주세요
------------------------------------------------------------
답변: 테슬라 Cybertruck은 2019년 11월에 처음 발표된 풀사이즈 픽업 트럭입니다. 이 차량은 후륜 구동, 듀얼 모터 전륜 구동, 트리 모터 전륜 구동의 세 가지 모델로 제공됩니다. Cybertruck의 배송은 2023년 11월에 시작되었습니다. (출처: 문서 1, 문서 2, 문서 3)
📊 검색된 문서 수: 3


질문 2: 리비안 R1T와 테슬라 Cybertruck의 차이점은 무엇인가요?
------------------------------------------------------------
답변: 제공된 문서에서는 리비안 R1T에 대한 정보가 없으므로, 리비안 R1T와 테슬라 Cybertruck의 차이점에 대한 구체적인 비교를 제공할 수 없습니다. 그러나 테슬라 Cybertruck에 대한 정보는 다음과 같습니다:

- **모델명:** Cybertruck
- **발표 연도:** 2019년 11월
- **모델 사양:** 후륜 구동, 듀얼 모터 전륜 구동, 트리 모터 전륜 구동의 세 가지 모델이 제공됩니다.
- **배송 시작:** 2023년 11월에 Cybertruck 배송이 시작되었습니다.

제공된 문서에서는 리비안 R1T에 대한 정보가 없으므로, 해당 정보를 찾을 수 없습니다. (출처: data\테슬라_KR.md)
📊 검색된 문서 수: 2


질문 3: 테슬라에서 생산하는 트럭 모델들을 모두 나열해주세요
------------------------------------------------------------
답변: 테슬라에서 생산하는 트럭 모델은 다음과 같습니다:

1. **Cybertruck**
   - 발표 연도: 2019년
   - 모델 사양: 후륜 구동, 듀얼 모터 전륜 구동, 트리 모터 전륜 구동의 세 가지 모델 제공

2. *